In [8]:
import cv2
import numpy as np
import os

In [9]:
import cv2
import numpy as np

def detect_icons_feature_based(image_path, template_paths, threshold, output_folder, icon_name):
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: Unable to load the image from '{image_path}'.")
        return
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    img_detected = img.copy()
    detected_points = []

    # Initialize ORB detector
    orb = cv2.ORB_create()

    # Iterate over each template
    for template_path in template_paths:
        template = cv2.imread(template_path, 0)
        if template is None:
            print(f"Error: Unable to load the template '{template_path}'.")
            continue
        
        # Find keypoints and descriptors in the floor plan and the template
        kp1, des1 = orb.detectAndCompute(img_gray, None)
        kp2, des2 = orb.detectAndCompute(template, None)

        # Ensure descriptors are the correct type (CV_8U)
        des1 = np.asarray(des1, dtype=np.uint8) if des1 is not None else None
        des2 = np.asarray(des2, dtype=np.uint8) if des2 is not None else None

        # Check if descriptors are not None and of the correct type
        if des1 is None or des2 is None:
            print("Error: Descriptors are None for one of the images.")
            continue

        # Use BFMatcher to match descriptors
        bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
        matches = bf.match(des1, des2)

        # Sort matches based on distance
        matches = sorted(matches, key=lambda x: x.distance)

        # Define a threshold for good matches (you can tune this value)
        good_matches = [m for m in matches if m.distance < threshold]
        
        # If enough good matches are found, proceed
        if len(good_matches) > 5:
            # Extract the matching keypoints
            src_pts = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
            dst_pts = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

            # Find homography (transformation matrix)
            M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

            # Get the size of the template
            h, w = template.shape
            pts = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1, 1, 2)
            dst = cv2.perspectiveTransform(pts, M)

            # Draw the bounding box around the matched region
            img_detected = cv2.polylines(img_detected, [np.int32(dst)], True, (0, 255, 0), 3)
            detected_points.append(dst)
    
    # Show and save the final result
    cv2.imshow("Detected Icons", img_detected)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    filename = f"{icon_name}_detected.jpg"
    filepath = os.path.join(output_folder, filename)

    # Ensure output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    cv2.imwrite(filepath, img_detected)
    print(f"Result saved to {filepath}")

In [10]:
output_folder = 'C:/Users/pis05408.PINNACLE/Desktop/Suraj/ObjectDetectionFlorPlan/Test'
os.makedirs(output_folder, exist_ok=True)
print(f'output_folder: {output_folder}')

output_folder: C:/Users/pis05408.PINNACLE/Desktop/Suraj/ObjectDetectionFlorPlan/Test


In [11]:
# Load your target image and templates
image_path = 'data/sample.jpg'
# template_paths = ['data/test.jpg', 'data/test1.jpg', 'data/test1.jpg', 'data/test2.jpg', 'data/test3.jpg', 'data/test4.jpg', 'data/test5.jpg', 'data/test6.jpg']
template_paths = ['data/test.jpg']

# Read the images
image = cv2.imread(image_path)
templates = [cv2.imread(template_path) for template_path in template_paths]

In [12]:
# Define scales and angles
# scales = np.linspace(0.5, 1.2, num=10)  # Adjust scale range and steps as needed
scales = np.arange(0.5, 1.2, 0.1)

# Define threshold
threshold = 0.7  # Adjust based on experimentation

icon_name = 'testorb'

# Call the detection function
detect_icons_feature_based(image_path, template_paths, threshold, output_folder, icon_name)

Error: Descriptors are None for one of the images.
Result saved to C:/Users/pis05408.PINNACLE/Desktop/Suraj/ObjectDetectionFlorPlan/Test\testorb_detected.jpg


In [13]:
# def rotate_image(image, angle):
#     (h, w) = image.shape[:2]
#     center = (w / 2, h / 2)
#     M = cv2.getRotationMatrix2D(center, angle, 1.0)
#     cos = np.abs(M[0, 0])
#     sin = np.abs(M[0, 1])
#     new_w = int((h * sin) + (w * cos))
#     new_h = int((h * cos) + (w * sin))
#     M[0, 2] += (new_w / 2) - center[0]
#     M[1, 2] += (new_h / 2) - center[1]
#     rotated = cv2.warpAffine(image, M, (new_w, new_h))
#     return rotated

# def multi_scale_and_angle_template_matching(image, templates, threshold=0.7):
#     bounding_boxes = []

#     for template in templates:
#         h, w = template.shape[:2]
        
#         # Loop over scales
#         for scale in np.linspace(0.5, 1.5, num=10):  # Adjust scale range as needed
#             resized_template = cv2.resize(template, (int(w * scale), int(h * scale)))
#             res_h, res_w = resized_template.shape[:2]

#             # Skip if resized template is larger than the image
#             if res_h > image.shape[0] or res_w > image.shape[1]:
#                 continue

#             # Loop over angles
#             for angle in [90, 180, 270]:  # Adjust angle step as needed
#                 M = cv2.getRotationMatrix2D((res_w / 2, res_h / 2), angle, 1.0)
#                 rotated_template = cv2.warpAffine(resized_template, M, (res_w, res_h))

#                 # Template matching
#                 result = cv2.matchTemplate(image, rotated_template, cv2.TM_CCOEFF_NORMED)
#                 loc = np.where(result >= threshold)

#                 # Record all locations where matches are found
#                 for pt in zip(*loc[::-1]):  # Switch x and y coordinates
#                     bounding_boxes.append((pt[0], pt[1], res_w, res_h))  # (x, y, width, height)

#     return bounding_boxes

# def non_max_suppression(boxes, overlapThresh=0.7):
#     if len(boxes) == 0:
#         return []

#     boxes = np.array(boxes)
#     pick = []

#     x1 = boxes[:, 0]
#     y1 = boxes[:, 1]
#     x2 = boxes[:, 0] + boxes[:, 2]
#     y2 = boxes[:, 1] + boxes[:, 3]

#     area = (x2 - x1) * (y2 - y1)
#     idxs = np.argsort(y2)

#     while len(idxs) > 0:
#         last = len(idxs) - 1
#         i = idxs[last]
#         pick.append(i)

#         xx1 = np.maximum(x1[i], x1[idxs[:last]])
#         yy1 = np.maximum(y1[i], y1[idxs[:last]])
#         xx2 = np.minimum(x2[i], x2[idxs[:last]])
#         yy2 = np.minimum(y2[i], y2[idxs[:last]])

#         w = np.maximum(0, xx2 - xx1)
#         h = np.maximum(0, yy2 - yy1)
#         overlap = (w * h) / area[idxs[:last]]

#         idxs = np.delete(idxs, np.concatenate(([last], np.where(overlap > overlapThresh)[0])))

#     return boxes[pick].astype("int")

In [14]:
# # Perform multi-scale and multi-angle template matching
# bounding_boxes = multi_scale_and_angle_template_matching(image, templates)

# # Apply non-maxima suppression to reduce overlapping boxes
# filtered_boxes = non_max_suppression(bounding_boxes)

# # Draw bounding boxes on the original image for visualization
# for (x, y, w, h) in filtered_boxes:
#     cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)


# # Create a filename based on scale and threshold
# output_folder = 'output'  # Specify your output folder
# os.makedirs(output_folder, exist_ok=True)  # Create output folder if it doesn't exist
# filename = f"detected_scale_1.jpg"
# filepath = os.path.join(output_folder, filename)

# # Save the detected image
# cv2.imwrite(filepath, image)

# # Print the count of detected items
# print(f'File Stored at: {filepath}\nfile_name: {filename}')

# # Show the result
# cv2.imshow("Detected Icons", image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()